To make a kernel with a siphon virtualenv from the terminal:

```
conda create -n siphon siphon
conda activate siphon
conda install ipykernel
python -m ipykernel install --user --name=siphon
```

Then select the `siphon` from the pulldown/window in Jupyter.

In [1]:
from siphon.simplewebservice.wyoming import WyomingUpperAir
from datetime import datetime

start_time = datetime(2023, 3, 1, 0)
end_time = datetime(2023, 6, 30, 0)

station = "43371" # Trivandrum, Thiruvananthapuram, 'VOTX'?

df = WyomingUpperAir.request_data(start_time, station) # returns Pandas dataframe
# attributes like units are lost???

In [2]:
df

,pressure,height,temperature,dewpoint,direction,speed,u_wind,v_wind,station,station_number,time,latitude,longitude,elevation,pw
0,1005.0,64,26.2,22.1,0.0,0.0,-0.000000,-0.000000e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
1,1000.0,109,25.8,22.8,30.0,2.0,-1.000000,-1.732051e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
2,925.0,793,22.0,17.0,80.0,4.0,-3.939231,-6.945927e-01,,43371,2023-03-01,8.48,76.95,64.0,38.48
3,900.0,1030,20.2,16.4,75.0,10.0,-9.659258,-2.588190e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
4,850.0,1523,16.4,15.3,75.0,15.0,-14.488887,-3.882286e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
5,848.0,1543,16.2,15.2,75.0,15.0,-14.488887,-3.882286e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
6,841.0,1614,15.6,15.0,80.0,15.0,-14.772116,-2.604723e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
7,804.0,1992,13.8,12.7,105.0,17.0,-16.420739,4.399924e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
8,753.0,2543,11.1,9.3,90.0,21.0,-21.000000,-1.285879e-15,,43371,2023-03-01,8.48,76.95,64.0,38.48
9,738.0,2712,10.3,8.2,110.0,23.0,-21.612930,7.866463e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48


In [15]:
# for column in df:
#     print( df[column].name )
#     print( df[column].values )

# df[column].shape[0]
df[df.columns[0:8]]

# skip column 8

# scalars
# df[df.columns[9]][0]

,pressure,height,temperature,dewpoint,direction,speed,u_wind,v_wind
0,1005.0,64,26.2,22.1,0.0,0.0,-0.000000,-0.000000e+00
1,1000.0,109,25.8,22.8,30.0,2.0,-1.000000,-1.732051e+00
2,925.0,793,22.0,17.0,80.0,4.0,-3.939231,-6.945927e-01
3,900.0,1030,20.2,16.4,75.0,10.0,-9.659258,-2.588190e+00
4,850.0,1523,16.4,15.3,75.0,15.0,-14.488887,-3.882286e+00
5,848.0,1543,16.2,15.2,75.0,15.0,-14.488887,-3.882286e+00
6,841.0,1614,15.6,15.0,80.0,15.0,-14.772116,-2.604723e+00
7,804.0,1992,13.8,12.7,105.0,17.0,-16.420739,4.399924e+00
8,753.0,2543,11.1,9.3,90.0,21.0,-21.000000,-1.285879e-15
9,738.0,2712,10.3,8.2,110.0,23.0,-21.612930,7.866463e+00


In [ ]:
# bad way... & broken

#from siphon.cdmr import Dataset
from siphon.ncss import Dataset
from io import StringIO

# Convert the DataFrame to CSV format in memory
csv_buffer = StringIO()
df.to_csv(csv_buffer, sep=',', index=False)
csv_buffer.getvalue().splitlines()[0].encode()
csv_buffer.getvalue().splitlines()[1:]

# Open a Dataset object and write the CSV data to it
nc = Dataset('sounding.nc', 'w', format='NETCDF4')
nc.createDimension('index', None)
nc.createVariable('header', 'S1', ('header',))
# nc.variables['header'][0] = csv_buffer.getvalue().splitlines()[0].encode()
# nc.variables['header'][0] = 
nc.createVariable('data', 'f8', ('index', 'header'))

for i, column in enumerate(df):
    # print( df[column].name )
    nc.variables['header'][i] = df[column].name
    # print( df[column].values )
    
for i, line in enumerate(csv_buffer.getvalue().splitlines()[1:]):
    #nc.variables['data'][i, :] = line.split(',') # can't autoconvert string to float
    line.split(',')
nc.close()


In [27]:
from netCDF4 import Dataset

# Open a new NetCDF file for writing
nc_file = Dataset('sounding.nc', mode='w')

# Set the attributes from df cols 9...
for column in df[df.columns[9:]]:
    nc_file.setncattr(df[column].name, df[column])

# or set all at once from a dict: setncatts(self,attdict)

# Define dimensions for pressure, temperature, etc. variables
time_dim = nc_file.createDimension('time', 1)
level_dim = nc_file.createDimension('level', df[df.columns[0]].shape[0])

units_list = [ 'hPa', 'm', 'C', 'C', 'degrees', 'm/s', 'm/s', 'm/s' ]
# Create variables for time, level, latitude, longitude, pressure, and temperature
for i, column in enumerate( df[columns[0:7]] ):
    # create the netcdf variable
    var = nc_file.createVariable(df[column].name, 'f4', ('time', 'level'))
    # write the data to the NetCDF file
    var[:] = df[column][:]
    # supply units attributes
    var.units = units_list[i]

# Close the NetCDF file
nc_file.close()

TypeError: illegal data type for attribute b'time', must be one of dict_keys(['S1', 'i1', 'u1', 'i2', 'u2', 'i4', 'u4', 'i8', 'u8', 'f4', 'f8']), got M8[ns]